# basic ARIMA

In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
import sklearn.metrics as skmetrics #For evaluation metrics

import matplotlib.pyplot as plt

import warnings                               
warnings.filterwarnings('ignore')

In [3]:


# Load your data
# Assuming your data is in a CSV file with a 'Date' column and a 'Temperature' column
data = pd.read_csv('data/preprocessed_data.csv', parse_dates=['ds'], index_col='ds')
data.head()


,index,city_name,y,temp_min,temp_max,rain_1h,clouds_all
ds,,,,,,,
2014-12-31,4,Valencia,270.475000,270.475000,270.475000,0.0,0.0
2015-01-01,9,Valencia,275.986987,275.986987,275.986987,0.0,0.0
2015-01-02,14,Valencia,276.784833,276.784833,276.784833,0.0,0.0
2015-01-03,19,Valencia,279.203802,279.203802,279.203802,0.0,2.5
2015-01-04,24,Valencia,279.260000,279.260000,279.260000,0.0,0.0


In [ ]:
# Hyperparameters for ARIMA
arima_order = (5, 1, 0)  # (p, d, q)
sarima_order = (

In [ ]:
# Fit ARIMA model
arima_model = ARIMA(data['y'], order=arima_order)
arima_model_fit = arima_model.fit()
forecast = arima_model_fit.forecast(365)

# Summary of the model
print(arima_model_fit.summary())

# Plot the fitted values against the original data
data['ARIMA_Fitted'] = arima_model_fit.fittedvalues
data[['Temperature', 'ARIMA_Fitted']].plot(figsize=(10, 6))
plt.show()


In [ ]:
def evaluate_models_cv(dataset, p_values, d_values, q_values, P_values, D_values, Q_values, m):
    K=3
    result = []
    best_rmse, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                for P in P_values:
                    for D in D_values:
                        for Q in Q_values:
                            trend_order = (p,d,q)
                            seasonal_order = (P,D,Q,m)
                            cfg = [(p,d,q), (P,D,Q,m)]
                            validation_size=4
                            train_size=len(dataset)-validation_size*K
                            rmse=0
                            for k in range(0,3):
                                train, test = dataset[0:train_size+k*validation_size],dataset[train_size+k*validation_size:train_size+(k+1)*validation_size]
                                model = ARIMA(train, order=trend_order, seasonal_order=seasonal_order) #each step: rolling one step forward to update the train se
                                model_fit = model.fit()
                                predictions=model_fit.forecast(validation_size)
                                rmse = rmse+np.sqrt(skmetrics.mean_squared_error(test, predictions))
                            rmse_avg=rmse/3
                            result.append((cfg, rmse_avg))
                            if rmse_avg < best_rmse:
                                best_rmse, best_cfg = rmse_avg, cfg
                                print('ARIMA%s RMSE=%.3f' % (cfg,rmse_avg))
    print('Best SARIMA%s RMSE=%.3f' % (best_cfg, best_rmse))

In [ ]:
# order selection
p=[0,1,2]
d=[1,2]
q=[0,1,2]
P=[0,1,2]
D=[1]
Q=[0,1,2]
m=4

In [ ]:
evaluate_models_cv(df, p,d,q,P,D,Q,m)